In [ ]:
## فراخوانی کتابخانه های مربوط به خواندن فایل و پردازش داده ها
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt
# !pip install jdatetime
## فراخوانی دیگر کتابخوانه های مورد نیاز
from google.colab import drive
import jdatetime
from datetime import datetime
drive.mount('/content/drive')
## خوادن فایل
data = "/content/drive/MyDrive/Colab Notebooks/Divar.csv"
data_set = pd.read_csv(data, encoding='utf-8')
# data_set

خواندن فایل ها و پردازش داده ها

In [ ]:
# جدا کردن ستون های مد نظر برای پردازش
file = data_set.iloc[:, 0:9]
# حذف ستون ها
clean_df = file.drop(["description","title","user_type","Unnamed: 0"], axis=1)
# تبدیل تاریخ میلادی به شمسی و ایجاد یک ستون جدید به نام سال ثبت آگهی
clean_df["created_at_month"] = pd.to_datetime(clean_df["created_at_month"])
clean_df["created_at_year"] = clean_df["created_at_month"].apply(
    lambda x: jdatetime.date.fromgregorian(date=x.date()).year
)
clean_df["created_at_month"] = clean_df["created_at_month"].apply(
    lambda x: jdatetime.date.fromgregorian(date=x.date()).month
)
##### تغیر در طول و عرض جغرافیایی با utm
#####(کد نوشته شده توسط کوروش معماری مسئول رسیدگی به ستون های 23 الی 51)
x_longitude = data_set["location_longitude"]
y_latitude = data_set["location_latitude"]
from pyproj import Transformer
# 2) تعریف ترنسفورمر (WGS84 -> UTM Zone 39N)
transformer = Transformer.from_crs("epsg:4326", "epsg:32639", always_xy=True)
# 3) تبدیل مختصات
utm_easting, utm_northing = transformer.transform(x_longitude.values, y_latitude.values)
# 4) اضافه کردن ستون‌های جدید
data_set["UTM_Easting"]  = utm_easting
data_set["UTM_Northing"] = utm_northing
utm_df = data_set.copy()
# 1) utm mean
neigh_means = utm_df.groupby('neighborhood_slug')[['UTM_Easting','UTM_Northing']].transform('mean')
# 2) filling bu mean
utm_df['UTM_Easting']  = utm_df['UTM_Easting'].fillna(neigh_means['UTM_Easting'])
utm_df['UTM_Northing'] = utm_df['UTM_Northing'].fillna(neigh_means['UTM_Northing'])
city_means = utm_df.groupby('city_slug')[['UTM_Easting','UTM_Northing']].transform('mean')
# 2) filling by city mean
utm_df['UTM_Easting']  = utm_df['UTM_Easting'].fillna(city_means['UTM_Easting'])
utm_df['UTM_Northing'] = utm_df['UTM_Northing'].fillna(city_means['UTM_Northing'])
# اندیس‌ها
i_shiraz = 676502
i_tehran = 292674
# میانگین شیراز
shiraz_mean = utm_df.loc[utm_df['city_slug'] == 'shiraz', ['UTM_Easting','UTM_Northing']].mean()
# میانگین تهران
tehran_mean = utm_df.loc[utm_df['city_slug'] == 'tehran', ['UTM_Easting','UTM_Northing']].mean()
# جایگزینی NaNها
utm_df.loc[i_shiraz, ['UTM_Easting','UTM_Northing']] = shiraz_mean
utm_df.loc[i_tehran, ['UTM_Easting','UTM_Northing']] = tehran_mean
#### تبدیل عددی داده ها
lable = ["cat2_slug","cat3_slug","city_slug","neighborhood_slug"]
for col in lable:
    unique_vals = clean_df[col].dropna().unique()
    mapping = {val: i for i, val in enumerate(unique_vals, start=1)}
    clean_df[col] = clean_df[col].astype("category").cat.codes
#### پر کردن neighborhood_slug
clean_df = pd.concat([clean_df,utm_df["UTM_Easting"],utm_df["UTM_Northing"]],axis=1)
clean_df['neighborhood_slug'] = clean_df['neighborhood_slug'].replace(-1,np.nan)
knn_features = clean_df[["city_slug","UTM_Easting","UTM_Northing","neighborhood_slug"]]
imputer = KNNImputer(n_neighbors=2)
imputed = imputer.fit_transform(knn_features)
imputed
clean_df["neighborhood_slug"] =imputed[:,3]
# ذخیره تغیرات اعمال شده روی داده ها در یک فایل جدید
output_path = "/content/drive/MyDrive/Colab Notebooks/Divar_cleaned.csv"
clean_df.to_csv(output_path, index=False, encoding='utf-8')
